In [5]:
!uv pip install langchain tiktoken rapidocr-onnxruntime python-dotenv langchain-community langchain-groq

Using Python 3.12.7 environment at: C:\LLMOPS\.venv
Resolved 72 packages in 1.06s
Installed 4 packages in 90ms
 + distro==1.9.0
 + groq==0.37.1
 + langchain-groq==1.1.2
 + sniffio==1.3.1


In [1]:
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Data Ingestion

In [2]:
from langchain_community.document_loaders.text import TextLoader

loader = TextLoader("C:/LLMOPS/data/AgenticAI.txt", encoding="utf-8")
documents = loader.load()

c:\LLMOPS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
documents[0].page_content[:100] # print first 100 characters

'FIRST LOOK: Cadillac reveal their debut livery for 2026 F1 season during Super Bowl\nCadillac have be'

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

In [6]:
text_chunks = text_splitter.split_documents(documents)

In [7]:
text_chunks

[Document(metadata={'source': 'C:/LLMOPS/data/AgenticAI.txt'}, page_content='FIRST LOOK: Cadillac reveal their debut livery for 2026 F1 season during Super Bowl\nCadillac have become the latest team to unveil the livery for their 2026 challenger.'),
 Document(metadata={'source': 'C:/LLMOPS/data/AgenticAI.txt'}, page_content='Cadillac have unveiled their livery for the team’s debut season in Formula 1, with the design revealed during a television advert at the Super Bowl.'),
 Document(metadata={'source': 'C:/LLMOPS/data/AgenticAI.txt'}, page_content='Amid the sporting action at the annual championship game of the National Football League (NFL) – this year held in Santa Clara, California and played between the New England Patriots and the Seattle'),
 Document(metadata={'source': 'C:/LLMOPS/data/AgenticAI.txt'}, page_content='and the Seattle Seahawks – Cadillac showcased the colours that will adorn their car in 2026.'),
 Document(metadata={'source': 'C:/LLMOPS/data/AgenticAI.txt'}, page_c

In [23]:
!uv pip install faiss-cpu

Using Python 3.12.7 environment at: C:\LLMOPS\.venv
Resolved 3 packages in 1.00s
Installed 1 package in 512ms
 + faiss-cpu==1.13.2


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [9]:
embeddings = HuggingFaceEmbeddings()

C:\Users\NATHAN\AppData\Local\Temp\ipykernel_29408\3655315981.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\NATHAN\AppData\Local\Temp\ipykernel_29408\3655315981.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 999.31it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2

In [30]:
!uv pip install sentence-transformers

Using Python 3.12.7 environment at: C:\LLMOPS\.venv
Resolved 35 packages in 1.49s
Prepared 2 packages in 447ms
Installed 20 packages in 8.91s
 + click==8.3.1
 + filelock==3.20.3
 + fsspec==2026.2.0
 + hf-xet==1.2.0
 + huggingface-hub==1.4.1
 + jinja2==3.1.6
 + joblib==1.5.3
 + markupsafe==3.0.3
 + networkx==3.6.1
 + safetensors==0.7.0
 + scikit-learn==1.8.0
 + scipy==1.17.0
 + sentence-transformers==5.2.2
 + setuptools==82.0.0
 + shellingham==1.5.4
 + threadpoolctl==3.6.0
 + tokenizers==0.22.2
 + torch==2.10.0
 + transformers==5.1.0
 + typer-slim==0.21.1


In [10]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [11]:
vectorstore

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
# Perform similarity search
query = "What is the release date of the new Cadillac?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}")
    print(doc.page_content)
    print("-"*50)

Document 1
FIRST LOOK: Cadillac reveal their debut livery for 2026 F1 season during Super Bowl
Cadillac have become the latest team to unveil the livery for their 2026 challenger.
--------------------------------------------------
Document 2
Cadillac have unveiled their livery for the team’s debut season in Formula 1, with the design revealed during a television advert at the Super Bowl.
--------------------------------------------------
Document 3
Cadillac’s driver line-up for their first season also features Zhou Guanyu as a reserve driver, while former IndyCar star Colton Herta will hold a testing role.
--------------------------------------------------
Document 4
and the Seattle Seahawks – Cadillac showcased the colours that will adorn their car in 2026.
--------------------------------------------------


In [36]:
!uv pip install langchain_core

Using Python 3.12.7 environment at: C:\LLMOPS\.venv
Audited 1 package in 13ms


In [14]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:"""

In [15]:
prompt = ChatPromptTemplate.from_template(template)

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:"), additional_kwargs={})])

In [17]:
from langchain_core.output_parsers.string import StrOutputParser

In [18]:
output_parser = StrOutputParser()

In [19]:
from langchain_groq import ChatGroq

model = ChatGroq(model_name="qwen/qwen3-32b")

In [20]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [21]:
rag_chain.invoke("What is the Cadillac's debut livery for 2026 F1 season?")

'<think>\nOkay, so the user is asking about Cadillac\'s debut livery for the 2026 F1 season. Let me check the provided documents.\n\nFirst, the context mentions that Cadillac revealed their livery during the Super Bowl. The documents talk about a "FIRST LOOK" and that they unveiled the design during a TV advert at the Super Bowl. Also, they mentioned showcasing the colors for their 2026 car. Another part says the car had a special livery with founding names during the Barcelona Shakedown. \n\nWait, the user is asking specifically about the debut livery. The key points are the Super Bowl reveal and the colors. The Barcelona Shakedown had a special livery with names, but maybe that\'s a different event. The main debut was during the Super Bowl. The answer should mention the Super Bowl as the event where they showed the livery first. Also, the livery colors were showcased there. The documents don\'t specify exact colors but mention they revealed the design during that event. So the answer